In [247]:
# Mounting your personal Gdrive to the Colab notebook
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Reading
d = pd.read_csv('/gdrive/My Drive/CDSS-Hackathon-21/daily_aqi_2020_cali_wf.csv', index_col=0)
d['State.Code'] = d['State.Code'].astype(str).str.zfill(2)
d['County.Code'] = d['County.Code'].astype(str).str.zfill(3)

In [ ]:
pd.unique(d['start_date'])

array(['2020-08-16', nan, '2020-08-17', '2020-09-07', '2020-09-04',
       '2020-07-26', '2020-09-06', '2020-08-18', '2020-08-19'],
      dtype=object)

In [ ]:
# rolling 7 day with label at the center
d['7_ave'] = d['AQI'].rolling(window=7, center = True).mean()

In [ ]:
# get rid of outlier Humbolt:
d = d [d['County.Code']	!= '023']

In [ ]:
d_drop = d[d['day_diff']>=0]
d_drop['AQI_drop'] = d_drop[d_drop['7_ave']>50].day_diff
d_drop = d_drop.groupby(['State.Name','county.Name', 'State.Code','County.Code', 'fire_name', 'start_date']).agg({'AQI_drop':'min'}).reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
d[(d['fire_name'] == 'Red Salmon') & (d['day_diff']>0)]

,State.Name,county.Name,State.Code,County.Code,Date,AQI,Category,Defining.Parameter,Defining.Site,Number.of.Sites.Reporting,fire_name,start_date,day_diff,7_ave
3766,California,Humboldt,06,023,2020-07-27,18,Good,Ozone,06-023-1004,1,Red Salmon,2020-07-26,1.0,17.571429
3767,California,Humboldt,06,023,2020-07-28,11,Good,Ozone,06-023-1004,1,Red Salmon,2020-07-26,2.0,17.285714
3768,California,Humboldt,06,023,2020-07-29,8,Good,PM2.5,06-023-1004,1,Red Salmon,2020-07-26,3.0,15.428571
3769,California,Humboldt,06,023,2020-07-30,17,Good,Ozone,06-023-1004,1,Red Salmon,2020-07-26,4.0,14.285714
3770,California,Humboldt,06,023,2020-07-31,16,Good,Ozone,06-023-1004,1,Red Salmon,2020-07-26,5.0,14.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,California,Humboldt,06,023,2020-12-27,23,Good,Ozone,06-023-1004,1,Red Salmon,2020-07-26,154.0,36.142857
3920,California,Humboldt,06,023,2020-12-28,30,Good,PM10,06-023-1004,1,Red Salmon,2020-07-26,155.0,36.142857
3921,California,Humboldt,06,023,2020-12-29,67,Moderate,PM2.5,06-023-1004,1,Red Salmon,2020-07-26,156.0,42.000000
3922,California,Humboldt,06,023,2020-12-30,30,Good,Ozone,06-023-1004,1,Red Salmon,2020-07-26,157.0,44.857143


In [ ]:
d_viz = d[d['day_diff']>=0].merge(d_drop, on = ['State.Name','county.Name', 'State.Code','County.Code', 'start_date'], how = 'left')
d_viz = d_viz[d_viz['day_diff'] >= d_viz['AQI_drop']]

In [ ]:
d_viz = d_viz[d_viz['7_ave']<50].groupby(['State.Name','county.Name', 'State.Code','County.Code', 'fire_name','start_date']).agg({'day_diff':'min'}).reset_index()

In [ ]:
d_viz[d_viz['county.Name'] =='Los Angeles']
#d_viz.to_csv('/gdrive/My Drive/CDSS-Hackathon-21/viz_fire.csv',index=False, encoding='utf-8-sig') 

,State.Name,county.Name,State.Code,County.Code,Date,AQI,Category,Defining.Parameter,Defining.Site,Number.of.Sites.Reporting,fire_name,start_date,day_diff,7_ave,AQI_drop
1049,California,Los Angeles,06,037,2020-09-06,214,Very Unhealthy,Ozone,06-037-1103,17,Bobcat,2020-09-06,0.0,179.000000,0.0
1050,California,Los Angeles,06,037,2020-09-07,161,Unhealthy,PM2.5,06-037-9033,15,Bobcat,2020-09-06,1.0,172.857143,0.0
1051,California,Los Angeles,06,037,2020-09-08,127,Unhealthy for Sensitive Groups,PM2.5,06-037-9033,16,Bobcat,2020-09-06,2.0,169.857143,0.0
1052,California,Los Angeles,06,037,2020-09-09,95,Moderate,PM2.5,06-037-0002,17,Bobcat,2020-09-06,3.0,161.857143,0.0
1053,California,Los Angeles,06,037,2020-09-10,158,Unhealthy,PM2.5,06-037-0016,16,Bobcat,2020-09-06,4.0,154.857143,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,California,Los Angeles,06,037,2020-12-27,67,Moderate,PM2.5,06-037-1103,16,Bobcat,2020-09-06,112.0,73.857143,0.0
1162,California,Los Angeles,06,037,2020-12-28,51,Moderate,PM2.5,06-037-1302,16,Bobcat,2020-09-06,113.0,81.428571,0.0
1163,California,Los Angeles,06,037,2020-12-29,64,Moderate,PM2.5,06-037-1302,17,Bobcat,2020-09-06,114.0,80.857143,0.0
1164,California,Los Angeles,06,037,2020-12-30,76,Moderate,PM2.5,06-037-1302,16,Bobcat,2020-09-06,115.0,75.428571,0.0


In [252]:
!pip install geopandas


     |████████████████████████████████| 994 kB 7.4 MB/s 
     |████████████████████████████████| 15.4 MB 38 kB/s 
     |████████████████████████████████| 6.3 MB 13.9 MB/s 


In [253]:
import geopandas as gpd

In [255]:
d = gpd.read_file('/gdrive/My Drive/CDSS-Hackathon-21/FIre_2020')


In [257]:
d['centroid'] = d.centroid

In [260]:
d.columns

Index(['OBJECTID', 'YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME',
       'INC_NUM', 'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC',
       'GIS_ACRES', 'C_METHOD', 'OBJECTIVE', 'FIRE_NUM', 'Shape_Leng',
       'Shape_Area', 'geometry', 'centroid'],
      dtype='object')

In [261]:
d_c = d[['OBJECTID', 'YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME',
       'INC_NUM', 'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC',
       'GIS_ACRES', 'C_METHOD', 'OBJECTIVE', 'FIRE_NUM', 'Shape_Leng',
       'Shape_Area', 'centroid']]

In [263]:
d_c = d_c.set_geometry('centroid')

In [264]:
d_c.to_file('/gdrive/My Drive/CDSS-Hackathon-21/fire_centroid',index=False, encoding='utf-8-sig') 

In [ ]:
# Standardizing Column names
# CO AQ
co_aq_data.columns = co_aq_data.columns.str.replace(' ', '_') # replacing spaces (' ') with  underscore.
co_aq_data.columns = co_aq_data.columns.str.replace('-', '_') # replacing hyphens ('-') with underscore
co_aq_data.columns= co_aq_data.columns.str.lower() # converting column names to lower case

In [ ]:
co_aq_data.columns= ['state_code', 'county_code', 'site_num', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter_name', 'sample_duration',
       'pollutant_standard', 'date_local', 'units_of_measure', 'event_type',
       'observation_count', 'observation_percent', 'arithmetic_mean',
       'lst_max_value', 'lst_max_hour', 'aqi', 'method_code', 'method_name',
       'local_site_name', 'address', 'state_name', 'county_name', 'city_name',
       'cbsa_name', 'date_of_last_change']

In [ ]:
co_aq_data.to_gbq(destination_table= 'aqi_data.co_aq_data_18_21', 
          project_id='cdss-datathon-327823', 
          if_exists='replace', 
          progress_bar=True)

1it [00:47, 47.99s/it]
